In [1]:
from utils.read_file import *
from utils.model_tool import *

In [2]:
QUESTION_PATH = "競賽資料集/dataset/preliminary/questions_example.json"

INSURANCE_PATH = "競賽資料集/reference/insurance"

FAQ_PATH = "競賽資料集/reference/faq/pid_map_content.json"
ANS_PATH = "競賽資料集/dataset/preliminary/ground_truths_example.json"
FINANCE_PATH = "競賽資料集/reference/finance"

In [3]:
question = read_json(QUESTION_PATH)["questions"]

In [4]:
# from huggingface_hub import snapshot_download

# # 下載模型到指定路徑
# local_model_path = "./models/bge-m3"
# snapshot_download(repo_id="BAAI/bge-m3", revision="main", local_dir=local_model_path)

In [5]:
from sentence_transformers import SentenceTransformer

# 加載模型
embbeded_model = SentenceTransformer("./models/bge-m3", device='cuda')

c:\Users\weiso131\anaconda3\envs\rag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


### 驗證faq資料集的正確率

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 每段的目標長度
    chunk_overlap=300  # 分段間的重疊字符數
)

FILE_NUM = 2000

def validate_finance(embbeded_model, question):
    ans = read_json(ANS_PATH)
    correct = 0
    half_correct = 0 #答案在前五

    already_read = ["" for _ in range(FILE_NUM)]

    for i in range(50, 100):
        src = np.array(question[i]["source"])
        document = read_target_insurance_pdf(FINANCE_PATH, src, already_read)
        tokens = []
        chunk_real_index = []
        for j in range(len(document)):
            text = document[j]

            if already_read[src[j]] == "":
                already_read[src[j]] = text

            chunks = text_splitter.split_text(text)
            tokens.extend(chunks)
            chunk_real_index.extend([j + k * 0 for k in range(len(chunks))])

                
        chunk_real_index=np.array(chunk_real_index)


        token_index, similarities = get_ans(embbeded_model, question[i], tokens, 10)
        real_index = chunk_real_index[token_index]
        all_predict = src[real_index]
        predict = all_predict[0]


        if (predict == ans["ground_truths"][i]["retrieve"]):
            correct += 1
        else:
            show_wrong_ans(i, predict, ans, token_index, \
                   src, real_index, similarities, chunk_real_index, tokens, "output_finance")

        if (ans["ground_truths"][i]["retrieve"] in all_predict):
            half_correct += 1

    print(f"acc: {correct / 50 * 100} %")
    print(f"in rank 5: {half_correct / 50 * 100} %")

### 驗證finance資料集的正確率

In [7]:
import os

if not os.path.exists("output_finance/"):
    os.makedirs("output_finance/")
validate_finance(embbeded_model, question)

C:\Users\weiso131\AppData\Roaming\Python\Python312\site-packages\transformers\models\xlm_roberta\modeling_xlm_roberta.py:371: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


qid: 58, predict: 112, ans: 209
qid: 62, predict: 1024, ans: 591
qid: 64, predict: 706, ans: 124
qid: 67, predict: 837, ans: 1021
qid: 69, predict: 545, ans: 981
qid: 70, predict: 298, ans: 490
qid: 75, predict: 178, ans: 71
qid: 79, predict: 912, ans: 65
qid: 82, predict: 838, ans: 831
qid: 86, predict: 525, ans: 189
qid: 90, predict: 761, ans: 710
qid: 93, predict: 321, ans: 660
qid: 99, predict: 693, ans: 372
acc: 74.0 %
in rank 5: 98.0 %
